In [ ]:
import sys
sys.path.append('../src')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [ ]:
from port_opt.hpr import HprAlgorithm
from utils import load_data

In [ ]:
# live data from our db
data_all_index = utils.load_all_data.load_all_data()
# to save data to pkl
# utils.load_all_data.save_all_data('../data/data_all_index.pkl', data_all_index)
# use pkl for notebook
# data_all_index = load_data.load_all_data_plk('../data/data_all_index.pkl')

In [ ]:
@widgets.interact(
    index=widgets.Dropdown(options=['ibex', 'dax', 'stoxx'], value='ibex', description='index:'),
    date=widgets.IntSlider(min=250, max=data_all_index['ibex'][2], step=10, value=1000, layout=widgets.Layout(width='50%')),
    n_hist=widgets.IntSlider(min=10, max=250*5, step=10, value=250, layout=widgets.Layout(width='50%')),
)
def plot(index, date, n_hist):
    sclose, bm, n_days = data_all_index[index]
    
    date_time = bm.index[date]
    date_time_start_data = bm.index[date-n_hist]
    idx_date = sclose.index.get_loc(date_time)
    
    price_data = sclose.iloc[idx_date-n_hist:idx_date]
    price_data = price_data.dropna(axis=1)
    
    returns = np.log(price_data).diff()
    returns = returns.dropna()
    
    mu = returns.mean()
    cov = returns.cov()
    corr = returns.corr()
    
    
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(25, 7))
    bm.plot(ax=ax1)
    ax1.legend()
    ax1.axvline(x=date_time, linewidth=4, color='r')
    ax1.axvspan(date_time_start_data, date_time, alpha=0.5, color='grey')

    
    
    hpr = HprAlgorithm(cov, corr)
    hpr.plot_dendrogram()
    
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(25, 7))
    
    sns.heatmap(corr, ax=ax1, annot=True, fmt=".1f")
    
    corr_sort = hpr.sort_corr()
    sns.heatmap(corr_sort, ax=ax2, annot=True, fmt=".1f")
    
    sns.clustermap(hpr.dist, method='single', center=0, linewidths=.75)
    
    sns.heatmap(hpr.dist, ax=ax2, annot=True, fmt=".1f")